In [ ]:
import pandas as pd
from helpers import *
from IPython.display import display as dp
import joblib
import warnings
import nltk
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download("stopwords")

warnings.filterwarnings("ignore", category=UserWarning)

### Project and Notebook Summary:
We have 3 goals for this project:

1. Loan accepted/ rejected classification 
2. Accepted loan grade prediction 
3. Accepted loan subgrade prediction 

`In this notebook we are going to prepare 2 dataframes for modeling:`

1. comp_df
3. acc_prepped_df

The first one is used for our first goal, loan acceptance classification, \
and the last one is used for our 2nd and 3rd goal, loan_grade and loan_subgrade, \
prediction.

The EDA and modeling part would be in the following notebooks. 

- 2_eda.ipynb
- 3_modeling.ipynb

### Load Data
load data efficiently using memory map and assign suitable data types.

In [2]:
# please download data from the source and put them in the data folder

acc_file = "./data/accepted_2007_to_2018Q4.csv"
rej_file = "./data/rejected_2007_to_2018Q4.csv"

acc_df = convert_large_csv_file_to_dataframe(acc_file)
rej_df = convert_large_csv_file_to_dataframe(rej_file)

In [4]:
bprint("accepted")
dp(acc_df.head(3))
print("data_shape: ")
acc_df.shape


accepted


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,<NA>,3600,3600,3600.0,36 months,13.99,123.03,C,C4,...,<NA>,<NA>,Cash,N,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,68355089,<NA>,24700,24700,24700.0,36 months,11.99,820.28,C,C1,...,<NA>,<NA>,Cash,N,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,68341763,<NA>,20000,20000,20000.0,60 months,10.78,432.66,B,B4,...,<NA>,<NA>,Cash,N,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


data_shape: 


(2260701, 151)

In [5]:
# format column name
rej_cols_name = rej_df.columns
rej_sluggify_cols_name_map = {i: to_slug(i) for i in rej_cols_name}
rej_df = rej_df.rename(columns=rej_sluggify_cols_name_map)

bprint("rejected")
dp(rej_df.head(3))
print("data_shape: ")
rej_df.shape


rejected


,amount_requested,application_date,loan_title,risk_score,debt_to_income_ratio,zip_code,state,employment_length,policy_code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693,10%,481xx,NM,4 years,0
1,1000.0,2007-05-26,Consolidating Debt,703,10%,010xx,MA,< 1 year,0
2,11000.0,2007-05-27,Want to consolidate my debt,715,10%,212xx,MD,1 year,0


data_shape: 


(27648741, 9)

📝 There's different number of features in loan accepted and loan rejected \
tables. As the first goal is to predict if the loan would get accepted or rejected \
we need to ensure that the two tables have the same features.

### Data Wrangling

📊 Prune features that's irrelevant to modeling in accepted dataframe

Discard features which are only available `after loan approval`. 

Note: As I have limited knowledge for loan application, I consulted ChatGPT as \
domain expert for pre--loan-approval and post-loan-approval features selection.

In [6]:
print(f"There are {len(modeling_related_features)-3} pre-loan-approval features")
acc_df = acc_df[modeling_related_features]

There are 104 pre-loan-approval features


📊 Prepare data for `loan-accepted and loan-rejected prediction`.

To predict whether a loan application is going to pass or not, get features \
that appear both in accepted and rejected tables.

As the feature names varies a bit in both tables, below is the mapping:

Corresponding features between loan-rejected and loan-accepted:

- `amount_requested` : `loan_amnt`
- application_date : no_corresponding_feature
- `loan_title`: `purpose`
- `risk_score` : `(fico_range_low + fico_range_high)/2`
- `debt_to_income_ratio` : `dti`
- `zip_code`: `zip_code`
- `state`: `addr_state`
- `employment_length` : `emp_length`
- policy_code : policy_code


> Features that are not highlighted are not going to be included in modeling, as \
> they either only appear in one table, or are direct spoiler to prediction result. \
> the rest of the features would be further inspected.

In [7]:
# average fico_range high and low data to use it as risk_score in loan_accepted table
fico_range_mid = (acc_df.fico_range_high + acc_df.fico_range_low) / 2
acc_df["fico_range_mid"] = fico_range_mid

acc_comparison_feats = [
    "loan_amnt",
    "purpose",
    "fico_range_mid",
    "dti",
    "zip_code",
    "addr_state",
    "emp_length",
]
acc_comp_df = acc_df.loc[:, acc_comparison_feats].copy()

# rej_df
rej_comparison_feats = [
    "amount_requested",
    "loan_title",
    "risk_score",
    "debt_to_income_ratio",
    "zip_code",
    "state",
    "employment_length",
]

rej_comp_df = rej_df.loc[:, rej_comparison_feats].copy()

acc_comp_df = acc_comp_df.rename(
    columns={
        acc_comparison_feats[i]: rej_comparison_feats[i]
        for i in range(len(rej_comparison_feats))
    }
)

rej_comp_df["accepted"] = 0
acc_comp_df["accepted"] = 1


bprint("accepted_comparison_df")
dp(acc_comp_df.head(3))

bprint("rejected_comparison_df")
dp(rej_comp_df.head(3))


accepted_comparison_df


,amount_requested,loan_title,risk_score,debt_to_income_ratio,zip_code,state,employment_length,accepted
0,3600,debt_consolidation,677.0,5.91,190xx,PA,10+ years,1
1,24700,small_business,717.0,16.06,577xx,SD,10+ years,1
2,20000,home_improvement,697.0,10.78,605xx,IL,10+ years,1



rejected_comparison_df


,amount_requested,loan_title,risk_score,debt_to_income_ratio,zip_code,state,employment_length,accepted
0,1000.0,Wedding Covered but No Honeymoon,693,10%,481xx,NM,4 years,0
1,1000.0,Consolidating Debt,703,10%,010xx,MA,< 1 year,0
2,11000.0,Want to consolidate my debt,715,10%,212xx,MD,1 year,0


📊 Unify feature data format and types in two tables:

1. cast `debt_to_income_ratio` to type float
2. process and cast `employment_length` to type float
- use 0.5 for < 1 year
- use 15 for 10+ years
3. classify `loan_title` into 14 categories

1. Remove percentile sign in `debt_to_income` column and cast type to float

In [8]:
rej_comp_df["debt_to_income_ratio"] = rej_comp_df.debt_to_income_ratio.apply(
    lambda i: i.split("%")[0]
).astype("float")

2. process and cast `employment_length` to type int

In [9]:
acc_emp_len = acc_comp_df.employment_length.fillna("missing")
acc_emp_len = acc_emp_len.apply(lambda i: emp_year_map[i])
acc_comp_df["employment_length"] = acc_emp_len

rej_emp_len = rej_comp_df.employment_length.fillna("missing")
rej_emp_len = rej_emp_len.apply(lambda i: emp_year_map[i])
rej_comp_df["employment_length"] = rej_emp_len

#### 3. Classify messy `loan_title` data in rejected table into 14 categories as in accepted table

The loan_title data in loan_rejected table is messy with different cases, slight \
grammatical differences, .... 

> `TfidfVectorizer` and `MultinomialNaiveBayes` are used to tokenize, vectorize, \
> and classify loan_title into 14 classes as in the loan_accepted table.

- Replace all null values with 'other' category in both tables

In [10]:
# accepted
acc_loan_title = acc_comp_df["loan_title"].fillna("other")
acc_loan_title = acc_loan_title.apply(lambda i: " ".join(i.split("_")))


# rejected
rej_loan_title = rej_comp_df.loan_title
rej_loan_title = rej_loan_title.fillna("other")
rej_loan_title = rej_loan_title.apply(lambda i: " ".join(i.split("_")))
rej_loan_title = rej_loan_title.replace(" ", "other").tolist()

- Format loan title for tokenization

In [13]:
# separate with space for tokenization
loan_title_dict = {id: i for id, i in enumerate(
    acc_loan_title.unique().tolist())}

bprint("loan_accepted table loan_title column: ")
print("14 categories")
dp(loan_title_dict)

bprint("loan_rejected table loan_title column: ")
print(f"{rej_comp_df.loan_title.nunique()} categories")


loan_accepted table loan_title column: 
14 categories


{0: 'debt consolidation',
 1: 'small business',
 2: 'home improvement',
 3: 'major purchase',
 4: 'credit card',
 5: 'other',
 6: 'house',
 7: 'vacation',
 8: 'car',
 9: 'medical',
 10: 'moving',
 11: 'renewable energy',
 12: 'wedding',
 13: 'educational'}


loan_rejected table loan_title column: 
73927 categories


- Define corpus and labels for 14 categories \
(check the `helpers/helper_data.py` file for X_title_train and y_title_train)

In [14]:
assert len(X_title_train) == len(y_title_train), "train test mismatch"

- Classify rejected loan_title

In [18]:
stop_words = set(stopwords.words("english"))
tfid_vectorizer = TfidfVectorizer(stop_words=list(stop_words))

# give 'other' class higher weight to collect all the unclassifiable loan_title
class_prior = [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.9, 0.7, 0.7, 0.7, 0.7]
mn_nb = MultinomialNB(class_prior=class_prior)

pipeline = Pipeline(
    steps=[("tfid_transform", tfid_vectorizer), ("multinomial_nb", mn_nb)]
)

pipeline.fit(X_title_train, y_title_train)

y_title_pred = pipeline.predict(rej_loan_title)
dp(pipeline)

Pipeline(steps=[('tfid_transform',
                 TfidfVectorizer(stop_words=['i', 'doesn', 'himself', 'hasn',
                                             'such', 'both', 'again', 'where',
                                             'this', 'from', 've', 'if', 'over',
                                             "weren't", 'in', 'nor', 'her',
                                             'you', 'ourselves', 'have',
                                             'further', 'now', 'all', 'most',
                                             'ain', 'a', 'under', "needn't",
                                             'will', "you'd", ...])),
                ('multinomial_nb',
                 MultinomialNB(class_prior=[0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7,
                                            0.7, 0.7, 0.9, 0.7, 0.7, 0.7,
                                            0.7]))])

In [20]:
title_classification_demo = list(zip(rej_loan_title[:15], y_title_pred[:15]))
bprint("Classification result demo: ")
title_classification_demo


Classification result demo: 


[('Wedding Covered but No Honeymoon', 'wedding'),
 ('Consolidating Debt', 'debt consolidation'),
 ('Want to consolidate my debt', 'debt consolidation'),
 ('waksman', 'other'),
 ('mdrigo', 'other'),
 ('Trinfiniti', 'other'),
 ('NOTIFYi Inc', 'other'),
 ('For Justin.', 'other'),
 ('title?', 'other'),
 ('timgerst', 'other'),
 ('need to consolidate', 'debt consolidation'),
 ('sixstrings', 'other'),
 ('bmoore5110', 'other'),
 ('MHarkins', 'other'),
 ('Moving', 'moving')]

- Replace loan_title with slug case category in both tables

In [21]:
# accepted
acc_comp_df["loan_title"] = acc_loan_title.apply(lambda i: "_".join(i.split(" ")))

# rejected
rej_comp_df["loan_title"] = pd.Series(y_title_pred).apply(
    lambda i: "_".join(i.split(" "))
)
bprint("accepted: ")
dp(acc_comp_df.head(3))

bprint("rejected: ")
dp(rej_comp_df.head(3))


accepted: 


,amount_requested,loan_title,risk_score,debt_to_income_ratio,zip_code,state,employment_length,accepted
0,3600,debt_consolidation,677.0,5.91,190xx,PA,15.0,1
1,24700,small_business,717.0,16.06,577xx,SD,15.0,1
2,20000,home_improvement,697.0,10.78,605xx,IL,15.0,1



rejected: 


,amount_requested,loan_title,risk_score,debt_to_income_ratio,zip_code,state,employment_length,accepted
0,1000.0,wedding,693,10.0,481xx,NM,4.0,0
1,1000.0,debt_consolidation,703,10.0,010xx,MA,0.5,0
2,11000.0,debt_consolidation,715,10.0,212xx,MD,1.0,0


#### Null, duplication, and anamolies handling for acc_comp_df and rej_comp_df

- Note: nulls won't be imputed here, it would be filled with 99999 and imputed \
in the modeling step to test out different imputation methods.

- Null detection

In [22]:
show = input_to_df(acc_comp_df.isnull().sum(), col_name="loan_accepted_null")
show["loan_rejected_null"] = input_to_df(rej_comp_df.isnull().sum()).col
dp(show)

,loan_accepted_null,loan_rejected_null
amount_requested,33,0
loan_title,0,0
risk_score,33,18497630
debt_to_income_ratio,1744,0
zip_code,34,293
state,33,22
employment_length,0,0
accepted,0,0


In loan_accepted table, there seems to be 33 rows with all-null vals, those \
rows can be dropped.

In [23]:
acc_comp_df = acc_comp_df.dropna(subset=["amount_requested"])
rej_comp_df = rej_comp_df.dropna(subset=["amount_requested"])

also drop null in zip_code and state.

In [24]:
acc_comp_df = acc_comp_df.dropna(subset=["zip_code"])
rej_comp_df = rej_comp_df.dropna(subset=["zip_code", "state"])

fillna with 9999

In [25]:
subset = ["risk_score", "debt_to_income_ratio"]
acc_comp_df.loc[:, subset] = acc_comp_df.loc[:, subset].fillna(99999)
rej_comp_df.loc[:, subset] = rej_comp_df.loc[:, subset].fillna(99999)

- Anamolies detection

📊 Anamolies in loan_accepted table and some observations:

In [26]:
acc_comp_df.describe()

,amount_requested,risk_score,debt_to_income_ratio,employment_length,accepted
count,2260667.0,2260667.0,2260667.0,2.260667e+06,2260667.0
mean,15046.935407,700.588315,94.494818,6.570165e+02,1.0
std,9190.245373,33.010812,2749.552787,2.462826e+03,0.0
min,500.0,612.0,-1.0,5.000000e-01,1.0
25%,8000.0,677.0,11.9,3.000000e+00,1.0
50%,12900.0,692.0,17.85,7.000000e+00,1.0
75%,20000.0,717.0,24.51,1.500000e+01,1.0
max,40000.0,847.5,99999.0,9.999000e+03,1.0


It is counterintuitive that debt_to_income_ratio (dti) is -1, as dti is positive \
float variable. \
I could not find clear explanation on why this occurs after some search.

let's inspect `debt_to_income_ratio == -1` rows in both tables:

In [27]:
acc_neg_dti = acc_comp_df.loc[acc_comp_df.debt_to_income_ratio == -1]
bprint("loan_accepted: ")
dp(acc_neg_dti)

rej_neg_dti = rej_comp_df.loc[rej_comp_df.debt_to_income_ratio == -1]
bprint("loan_rejected: ")
dp(rej_neg_dti)
print(f"There are {rej_neg_dti.shape[0]} negative dti rows in loan_rejected table")


loan_accepted: 


,amount_requested,loan_title,risk_score,debt_to_income_ratio,zip_code,state,employment_length,accepted
1014615,15000,debt_consolidation,672.0,-1.0,970xx,OR,5.0,1
1681348,17000,debt_consolidation,687.0,-1.0,471xx,IN,15.0,1



loan_rejected: 


,amount_requested,loan_title,risk_score,debt_to_income_ratio,zip_code,state,employment_length,accepted
10559,5000.0,car,99999,-1.0,203xx,DC,0.5,0
10565,25000.0,small_business,99999,-1.0,919xx,CA,2.0,0
10570,2000.0,medical,99999,-1.0,906xx,CA,1.0,0
10581,5000.0,other,99999,-1.0,833xx,ID,0.5,0
10624,23000.0,home_improvement,99999,-1.0,301xx,GA,5.0,0
...,...,...,...,...,...,...,...,...
27648711,8000.0,other,99999,-1.0,787xx,TX,5.0,0
27648715,40000.0,major_purchase,99999,-1.0,148xx,NY,9999.0,0
27648717,10000.0,other,99999,-1.0,531xx,WI,0.5,0
27648724,9000.0,debt_consolidation,99999,-1.0,856xx,AZ,0.5,0


There are 1202771 negative dti rows in loan_rejected table


📝 debt_to_income_ratio (dti) = -1 in loan_accepted and rejected tables:

There are only 2 rows of data with dti = -1 in loan_accepted table, while more \
than 1 million rows in loan_rejected table.

It seems like dti = -1 is either an error or a flag for unknown implication. \
As it occurs almost only in loan_rejected table, it is more likely that it's \
used as a flag than an error.

> Thus we would replace dti = -1 with 99999, this way, models like xgboost can \
handle the imputation for us.

In [28]:
acc_comp_df["debt_to_income_ratio"] = acc_comp_df.debt_to_income_ratio.replace(
    -1, 99999
)

rej_comp_df["debt_to_income_ratio"] = rej_comp_df.debt_to_income_ratio.replace(
    -1, 99999
)

Continue to inspect anomalies in loan rejected table.

In [29]:
rej_comp_df.describe()

,amount_requested,risk_score,debt_to_income_ratio,employment_length,accepted
count,27648448.0,27648448.0,2.764845e+07,2.764845e+07,27648448.0
mean,13133.22142,67109.212172,4.493571e+03,3.452346e+02,0.0
std,15009.671968,46760.560944,2.293290e+04,1.822369e+03,0.0
min,0.0,0.0,0.000000e+00,5.000000e-01,0.0
25%,4800.0,676.0,1.026000e+01,5.000000e-01,0.0
50%,10000.0,99999.0,2.223000e+01,5.000000e-01,0.0
75%,20000.0,99999.0,4.104000e+01,5.000000e-01,0.0
max,1400000.0,99999.0,5.000003e+07,9.999000e+03,0.0


📊 Anomolies in loan_rejected table and some discoveries:

1. There are some rows with `amount_requested = 0`, this should not occur as its \
meaningless or might be an error. 
> `amount_requested = 0` rows can be dropped.

2. The 75th percentile of employment length in loan_rejected table is 0.5 years ! \
This is the minimum we have in this column, these people also have less credit \
score (risk_score = 0) as they are in the beginnign of their career. 

- On the contrary, there's no row in loan_accepted table with risk_score = 0 
- Bank is not lending to people with no credit_score(risk_score).

In [30]:
zero_amount_requested = rej_comp_df.loc[rej_comp_df.amount_requested == 0]
bprint("zero amount requested")
dp(zero_amount_requested)


zero amount requested


,amount_requested,loan_title,risk_score,debt_to_income_ratio,zip_code,state,employment_length,accepted
531884,0.0,other,677,32.28,029xx,RI,0.5,0
594623,0.0,other,685,44.04,271xx,NC,0.5,0
816062,0.0,other,99999,99999.00,154xx,PA,0.5,0
816528,0.0,other,99999,99999.00,451xx,OH,0.5,0
820025,0.0,other,99999,99999.00,425xx,KY,0.5,0
...,...,...,...,...,...,...,...,...
27436134,0.0,other,99999,99999.00,301xx,GA,0.5,0
27456345,0.0,other,99999,99999.00,112xx,NY,0.5,0
27465211,0.0,other,99999,99999.00,015xx,MA,0.5,0
27496229,0.0,other,99999,99999.00,068xx,CT,0.5,0


drop zero amount requested data

In [31]:
rej_comp_df = rej_comp_df.drop(zero_amount_requested.index)

- Duplication detection

In [32]:
bprint("loan_accepted")
acc_dup = acc_comp_df.duplicated()
print(f"{acc_dup.sum()} duplications")

bprint("loan_rejected")
rej_dup = rej_comp_df.duplicated()
print(f"{rej_dup.sum()} duplications")


loan_accepted
296 duplications

loan_rejected
2971439 duplications


In [33]:
rej_comp_df.dtypes

amount_requested               Float64
loan_title                      object
risk_score                       Int64
debt_to_income_ratio           float64
zip_code                string[python]
state                   string[python]
employment_length              float64
accepted                         int64
dtype: object

Drop all the duplicated data

In [34]:
rej_comp_df = rej_comp_df.drop_duplicates()
acc_comp_df = acc_comp_df.drop_duplicates()

Concat two tables, serialize and export the cleaned dataframes.

In [35]:
comp_df = pd.concat([rej_comp_df, acc_comp_df])

joblib.dump(comp_df, "data/comp_df")
joblib.dump(rej_comp_df, "data/rej_comp_df")
joblib.dump(acc_comp_df, "data/acc_comp_df")

['data/acc_comp_df']

---

📊 Cleanup acc_df (the full loan_accepted df)

- Note: nulls won't be imputed here, it would be filled with 9999 and imputed \
in the modeling step to test out different imputation methods.

Drop all rows which any of the target variables is null

In [36]:
acc_df = acc_df.dropna(subset=["int_rate", "grade", "sub_grade"])

Inspect the non numerical columns and proccess accordingly

In [37]:
acc_df.select_dtypes(["string"]).head()

,grade,sub_grade,term,emp_title,emp_length,home_ownership,verification_status,issue_d,purpose,title,zip_code,addr_state,earliest_cr_line,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type,verification_status_joint
0,C,C4,36 months,leadman,10+ years,MORTGAGE,Not Verified,Dec-2015,debt_consolidation,Debt consolidation,190xx,PA,Aug-2003,w,Jan-2019,<NA>,Mar-2019,Individual,<NA>
1,C,C1,36 months,Engineer,10+ years,MORTGAGE,Not Verified,Dec-2015,small_business,Business,577xx,SD,Dec-1999,w,Jun-2016,<NA>,Mar-2019,Individual,<NA>
2,B,B4,60 months,truck driver,10+ years,MORTGAGE,Not Verified,Dec-2015,home_improvement,<NA>,605xx,IL,Aug-2000,w,Jun-2017,<NA>,Mar-2019,Joint App,Not Verified
3,C,C5,60 months,Information Systems Officer,10+ years,MORTGAGE,Source Verified,Dec-2015,debt_consolidation,Debt consolidation,076xx,NJ,Sep-2008,w,Feb-2019,Apr-2019,Mar-2019,Individual,<NA>
4,F,F1,60 months,Contract Specialist,3 years,MORTGAGE,Source Verified,Dec-2015,major_purchase,Major purchase,174xx,PA,Jun-1998,w,Jul-2016,<NA>,Mar-2018,Individual,<NA>


In [38]:
acc_df.select_dtypes(["string"]).isnull().sum()

grade                              0
sub_grade                          0
term                               0
emp_title                     166969
emp_length                    146907
home_ownership                     0
verification_status                0
issue_d                            0
purpose                            0
title                          23326
zip_code                           1
addr_state                         0
earliest_cr_line                  29
initial_list_status                0
last_pymnt_d                    2427
next_pymnt_d                 1345310
last_credit_pull_d                72
application_type                   0
verification_status_joint    2144938
dtype: int64

In [39]:
# cleanup more post_loan features
acc_df = acc_df.drop(
    columns=[
        "emp_title",
        "issue_d",
        "title",
        "last_credit_pull_d",
        "next_pymnt_d",
        "last_pymnt_d",
    ]
)

# drop 1 row of data with zip_code null
acc_df = acc_df.dropna(subset=["zip_code"])

Convert some numerical columns to ordinal dtype, and flag the nulls with 99999

In [40]:
# term
term_a = acc_df.term.apply(
    lambda i: 0 if i == " 36 months" else 1 if i == " 60 months" else 99999
)
acc_df["term"] = term_a

# emp_length
emp_len_a = acc_df.emp_length.fillna("missing")
emp_len_a = emp_len_a.apply(lambda i: emp_year_map[i])
acc_df["emp_length"] = emp_len_a

# verification_status
v_stat_map = {"Verified": 2, "Source Verified": 1, "Not Verified": 0}
verification_status_a = acc_df.verification_status.apply(
    lambda i: v_stat_map[i])
acc_df["verification_status"] = verification_status_a

# verification_status_joint
v_stat_map = {"Verified": 2, "Source Verified": 1,
              "Not Verified": 0, "missing": 99999}
verification_status_joint_a = acc_df.verification_status_joint.fillna(
    "missing")
verification_status_joint_a = verification_status_joint_a.apply(
    lambda i: v_stat_map[i])
acc_df["verification_status_joint"] = verification_status_joint_a

# earliest_cr_line: extract year
earliest_cr_line_a = acc_df.earliest_cr_line.fillna("month-99999")
earliest_cr_line_a = earliest_cr_line_a.apply(
    lambda i: i.split("-")[1]).astype("int")
acc_df["earliest_cr_line"] = earliest_cr_line_a

In [41]:
acc_df.select_dtypes(["string", "object"]).isnull().sum()

grade                  0
sub_grade              0
home_ownership         0
purpose                0
zip_code               0
addr_state             0
initial_list_status    0
application_type       0
dtype: int64

Fill the nulls in numerical columns with 99999

In [42]:
acc_df = acc_df.fillna(99999)

💡 Notebook Summary 

This is the end of the data preparation. 

We processed and selected data from the original loan_accepted and loan_rejected \
dataframes,handle duplications and anamolies, and fill all null values with 9999. 

Go to `2_project_eda.ipynb` for the next part.